In [1]:
from google.colab import drive
drive.mount("/drive")

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import KFold
import time
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")

Mounted at /drive


In [2]:
results_filename = "/drive/MyDrive/ACA/Projeto/Results_GridSearch_TVT/Results_KNN1.csv"
results_filename1 = "/drive/MyDrive/ACA/Projeto/Results_GridSearch_TVT/Results_KNN1.pkl"
def get_results():
    if os.path.exists(results_filename):
        print("Ficheiro existe")
        results = pd.read_pickle(results_filename1)
    else:
        print("Ficheiro nao existe")
        results = pd.DataFrame( columns=["Type", "Dataset", "Train Size", "Validation Size", "Test Size", "Train F1", "Train Recall", "Train Precision", "Train Accuracy", "Validation F1", "Validation Recall", "Validation Precision", "Validation Accuracy", "Test F1", "Test Recall", "Test Precision", "Test Accuracy", "Params"])
    return results
results = get_results()
results

Ficheiro nao existe


,Type,Dataset,Train Size,Validation Size,Test Size,Train F1,Train Recall,Train Precision,Train Accuracy,Validation F1,Validation Recall,Validation Precision,Validation Accuracy,Test F1,Test Recall,Test Precision,Test Accuracy,Params


In [3]:
datasets_folder = "/drive/MyDrive/ACA/Projeto/dataset/Clean/Train/"

files = os.listdir(datasets_folder)
print(files)
for filename in files:
    if (
        (results["Type"] == "KNN") &
        (results["Dataset"] == filename)
    ).any() == False:
        print(filename)
        df = pd.read_csv(datasets_folder + filename)

        if "Target" in list(df.columns):
            colunas_train = list(df.columns)
            colunas_train.remove("Target")
            
            X = list( df[colunas_train].values )
            y = df["Target"].values
        
        elif "attack_type" in list(df.columns):
            colunas_train = list(df.columns)
            colunas_train.remove("attack_type")

            X = list( df[colunas_train].values )
            y = df["attack_type"].values
        
        else: 
            print(df.columns)

        #==============================================================================================================
        #==== Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.15, shuffle= True)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0, test_size=0.175)

        print("\t",np.shape(X_train), "-", np.shape(X_val), np.shape(X_test) )
        #==============================================================================================================
        #==== Train
        
        # parameters
        parameters = {"n_neighbors": [3, 5, 7, 9, 11], 
                      "weights": ["uniform", "distance"],
                      "p": [0,1,2]}

        search = GridSearchCV(KNeighborsClassifier(), parameters, cv=3)
        search.fit(X_train, y_train) # train_model
        print("\t", search.best_params_)
        print("\t", search.best_score_)  

        pred_val = search.predict(X_train)

        f1_score_train = f1_score(pred_val, y_train, average='weighted')
        accuracy_score_train = accuracy_score(pred_val, y_train)
        recall_score_train = recall_score(pred_val, y_train, average='weighted')
        precision_score_train = precision_score(pred_val, y_train, average='weighted')  

        print("\tTrain")
        print("\tF1: ", f1_score_train)
        print("\tAccuracy: ", accuracy_score_train)
        print("\tRecall: ", recall_score_train)
        print("\tPrecision: ", precision_score_train)

        #==============================================================================================================
        #==== Validation  

        pred_val = search.predict(X_val)

        f1_score_val = f1_score(pred_val, y_val, average='weighted')
        accuracy_score_val = accuracy_score(pred_val, y_val)
        recall_score_val = recall_score(pred_val, y_val, average='weighted')
        precision_score_val = precision_score(pred_val, y_val, average='weighted')  

        print("\n\tValidation")
        print("\tF1: ", f1_score_val)
        print("\tAccuracy: ", accuracy_score_val)
        print("\tRecall: ", recall_score_val)
        print("\tPrecision: ", precision_score_val)
        #==============================================================================================================
        #==== Test

        pred_val = search.predict(X_test)

        f1_score_test = f1_score(pred_val, y_test, average='weighted')
        accuracy_score_test = accuracy_score(pred_val, y_test)
        recall_score_test = recall_score(pred_val, y_test, average='weighted')
        precision_score_test = precision_score(pred_val, y_test, average='weighted')  

        print("\n\tTest")
        print("\tF1: ", f1_score_test)
        print("\tAccuracy: ", accuracy_score_test)
        print("\tRecall: ", recall_score_test)
        print("\tPrecision: ", precision_score_test)

        #==============================================================================================================
        #=== Save Results
        results = get_results()
        results.loc[len(results)] = ["KNN", filename, np.shape(X_train), np.shape(X_val), np.shape(X_test), 
                                    f1_score_train, recall_score_train, precision_score_train, accuracy_score_train,
                                    f1_score_val, recall_score_val, precision_score_val, accuracy_score_val,
                                    f1_score_test, recall_score_test, precision_score_test, accuracy_score_test,
                                    search.best_params_]
        
        results.to_csv(results_filename)
        results.to_pickle(results_filename1)
    
    else:
        pass



    




['Outliers_NoCorr_NoST_PCA.csv', 'Outliers_NoCorr_NoST_NoPCA.csv', 'NoOutliers_NoCorr_NoST_PCA.csv', 'NoOutliers_NoCorr_NoST_NoPCA.csv', 'NoOutliers_NoCorr_ST_PCA.csv', 'NoOutliers_NoCorr_ST_NoPCA.csv', 'Outliers_NoCorr_ST_PCA.csv', 'Outliers_NoCorr_ST_NoPCA.csv']
Outliers_NoCorr_NoST_PCA.csv
	 (72901, 2) - (15465, 2) (15595, 2)
	 {'n_neighbors': 11, 'p': 2, 'weights': 'distance'}
	 0.9770922358131999
	Train
	F1:  0.9995893168388068
	Accuracy:  0.999588483011207
	Recall:  0.999588483011207
	Precision:  0.9995920874861851

	Validation
	F1:  0.9804497364469617
	Accuracy:  0.9802780472033624
	Recall:  0.9802780472033624
	Precision:  0.9807477595242668

	Test
	F1:  0.978339569536396
	Accuracy:  0.978069894196858
	Recall:  0.978069894196858
	Precision:  0.9787777694523729
Ficheiro nao existe
Outliers_NoCorr_NoST_NoPCA.csv
	 (72901, 40) - (15465, 40) (15595, 40)
	 {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
	 0.9879562775476676
	Train
	F1:  0.9995893168388068
	Accuracy:  0.999588483011